### Prepping Data Challenge: C&BSCo Mortgage Repayment (Week 32)
 
### Requirements
- Input the data
- Create a field for today (10th August 2022)
- Create a data field to show how much capital is paid off each month
- Create a data field to show how many months are needed to pay off the entire debt (whole months only)
- Create a field when the mortgages will be paid off by (Assuming a payment is to be made in August 2022)
- Create a row per month between now and when the mortgage is paid off showing:
  - How much is still to be paid off for that mortgage? Call this field 'Remaining Capital to Repay'
  - How much is still to be paid off for all mortgages? Call this field ' Capital Outstanding Total'
- Rename the date field 'Monthly Payment Date'
- Output the data

In [1]:
import pandas as pd
import numpy as np

In [2]:
#Input the data
df = pd.read_csv('WK32-input.csv')

In [3]:
df.head()

,Store,Capital Repayment Remaining,Monthly Payment,% of Monthly Repayment going to Capital
0,Lewisham,250000,1500,60
1,Wimbledon,350000,1900,55


In [4]:
#Create a field for today (10th August 2022)
df['today_date'] = pd.to_datetime(("10/08/2022"), format='%d/%m/%Y')

In [5]:
#Create a data field to show how much capital is paid off each month
df['mc_paid off'] = df['Monthly Payment'] * (df["% of Monthly Repayment going to Capital"]/100)

In [6]:
#Create a data field to show how many months are needed to pay off the entire debt (whole months only)
df['months_needed'] = (df["Capital Repayment Remaining"] / df["mc_paid off"]).round().astype(np.int64)

In [7]:
#Create a field when the mortgages will be paid off by (Assuming a payment is to be made in August 2022)
df['paid_off_date'] = np.where(df["Store"] == 'Lewisham', df["today_date"] + pd.DateOffset(months = 278), 
                                               df["today_date"] + pd.DateOffset(months = 335))

In [8]:
df.head()

,Store,Capital Repayment Remaining,Monthly Payment,% of Monthly Repayment going to Capital,today_date,mc_paid off,months_needed,paid_off_date
0,Lewisham,250000,1500,60,2022-08-10,900.0,278,2045-10-10
1,Wimbledon,350000,1900,55,2022-08-10,1045.0,335,2050-07-10


In [9]:
#Create a row per month between now and when the mortgage is paid off showing:
#Rename the date field 'Monthly Payment Date'
df['Monthly Payment Date'] = [pd.date_range(d, periods=p, freq=pd.DateOffset(months=1)) 
                              for d, p in zip(df["today_date"], df["months_needed"])]
df = df.explode('Monthly Payment Date')

In [10]:
df.head()

,Store,Capital Repayment Remaining,Monthly Payment,% of Monthly Repayment going to Capital,today_date,mc_paid off,months_needed,paid_off_date,Monthly Payment Date
0,Lewisham,250000,1500,60,2022-08-10,900.0,278,2045-10-10,2022-08-10
0,Lewisham,250000,1500,60,2022-08-10,900.0,278,2045-10-10,2022-09-10
0,Lewisham,250000,1500,60,2022-08-10,900.0,278,2045-10-10,2022-10-10
0,Lewisham,250000,1500,60,2022-08-10,900.0,278,2045-10-10,2022-11-10
0,Lewisham,250000,1500,60,2022-08-10,900.0,278,2045-10-10,2022-12-10


In [11]:
#How much is still to be paid off for that mortgage? Call this field 'Remaining Capital to Repay'
df['Remaining Capital to Repay'] = (df['Capital Repayment Remaining'] 
                                    - df.groupby('Store')['mc_paid off'].transform('cumsum'))

In [12]:
df['Capital Outstanding Total'] = (df.groupby('Monthly Payment Date')['Remaining Capital to Repay'].transform('sum'))

In [13]:
output = df[["Monthly Payment Date","Store",'Capital Outstanding Total','Remaining Capital to Repay']]

In [14]:
output.tail(10)

,Monthly Payment Date,Store,Capital Outstanding Total,Remaining Capital to Repay
1,2049-09-10,Wimbledon,9330.0,9330.0
1,2049-10-10,Wimbledon,8285.0,8285.0
1,2049-11-10,Wimbledon,7240.0,7240.0
1,2049-12-10,Wimbledon,6195.0,6195.0
1,2050-01-10,Wimbledon,5150.0,5150.0
1,2050-02-10,Wimbledon,4105.0,4105.0
1,2050-03-10,Wimbledon,3060.0,3060.0
1,2050-04-10,Wimbledon,2015.0,2015.0
1,2050-05-10,Wimbledon,970.0,970.0
1,2050-06-10,Wimbledon,-75.0,-75.0


In [15]:
#Output the data 
output.to_csv('wk32-output.csv', index=False)